In [ ]:
# Music Genre Classification Project

## Load and Preprocess Data


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the training data
train_data = pd.read_csv('data/train1.csv')
labels = train_data['label']
train_data = train_data.drop(['filename', 'label', 'length'], axis=1)

# Encode the genre labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(train_data, labels, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)


In [2]:
## Build and Compile Models
# Build the ANN model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')  # 10 output neurons for 10 genres
])

# Build the second ANN model
model2 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation=lambda x: tf.keras.activations.relu(x, alpha=0.01)),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')  # 10 output neurons for 10 genres
])

# Build the third ANN model
model3 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')  # 10 output neurons for 10 genres
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


C:\Users\Aaryan Shetty\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
## Train the Model
# Train the model
model.fit(X_train, y_train, epochs=56, batch_size=20, validation_data=(X_valid, y_valid))
# model.fit(X_train, y_train, epochs=39, batch_size=64, validation_data=(X_valid, y_valid))
# model.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_valid, y_valid))


Epoch 1/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3089 - loss: 1.9473 - val_accuracy: 0.5870 - val_loss: 1.1761
Epoch 2/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5530 - loss: 1.2557 - val_accuracy: 0.6504 - val_loss: 0.9979
Epoch 3/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6056 - loss: 1.1071 - val_accuracy: 0.6959 - val_loss: 0.8883
Epoch 4/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6470 - loss: 1.0113 - val_accuracy: 0.7215 - val_loss: 0.8328
Epoch 5/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6734 - loss: 0.9127 - val_accuracy: 0.7449 - val_loss: 0.7793
Epoch 6/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6909 - loss: 0.8903 - val_accuracy: 0.7499 - val_loss: 0.7507
Epoch 7/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7042 - loss: 0.8525 - val_accuracy: 0.7404 - val_loss: 0.7341
Epoch 8/56
360/360 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7054 - loss: 0.8252 - val_accuracy: 0.

In [4]:
## Evaluate the Model
# Evaluate the model
validation_loss, validation_accuracy = model.evaluate(X_valid, y_valid)
print(f'Validation Accuracy: {validation_accuracy}')


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8717 - loss: 0.3863
Validation Accuracy: 0.8749305009841919


In [6]:
## Predict and Save Results
# Load the test data
test_data = pd.read_csv('data/test1.csv')

# Extract 'id' column for later use in the submission
test_ids = test_data['id']

# Preprocess the test data (excluding 'id')
test_data = test_data.drop(['id', 'length'], axis=1)
test_data = scaler.transform(test_data)  # Standardize the features

# Make predictions using the ANN model
test_predictions = model.predict(test_data)
# test_predictions2 = model2.predict(test_data)
# test_predictions3 = model3.predict(test_data)

# Convert the predicted probabilities to labels
predicted_labels = np.argmax(test_predictions, axis=1)

# Create a DataFrame with the required format using the 'id' from the test data
submission_df = pd.DataFrame({'id': test_ids, 'label': predicted_labels})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('music_genre_predictions_final.csv', index=False)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [8]:
## Calculate Training Accuracy
# Load the training data
train_data = pd.read_csv('data/train1.csv')

# Preprocess the training data
labels = train_data['label']
train_data = train_data.drop(['filename', 'label', 'length'], axis=1)
labels = label_encoder.transform(labels)  # Encode labels

# Standardize the features
train_data = scaler.transform(train_data)

# Calculate the training accuracy
train_predictions = model.predict(train_data)
train_predictions = np.argmax(train_predictions, axis=1)

# Calculate the training accuracy
training_accuracy = np.mean(train_predictions == labels)
print(f'Training Accuracy: {training_accuracy}')


281/281 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step
Training Accuracy: 0.9423868312757202


In [11]:
## Compare Predictions with Ground Truth
import pandas as pd

# Load the ground truth labels for comparison
ground_truth = pd.read_csv('data/ground_truth.csv')  # Make sure you have a CSV file with the ground truth labels

# Load the test data
test_data = pd.read_csv('data/test1.csv')

# Extract 'id' column for later use in the submission
test_ids = test_data['id']

# Preprocess the test data (excluding 'id')
test_data = test_data.drop(['id', 'length'], axis=1)
test_data = scaler.transform(test_data)

# Make predictions using the ANN model
test_predictions = model.predict(test_data)

# Convert the predicted probabilities to labels
predicted_labels = np.argmax(test_predictions, axis=1)

print(list(predicted_labels))
print(ground_truth['label'])

# Compare the predicted labels to the ground truth labels
accuracy = (predicted_labels == ground_truth['label']).mean()

print(f'Accuracy with respect to ground truth: {accuracy}')


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
[9, 4, 7, 2, 0, 2, 6, 3, 3, 0, 3, 7, 6, 0, 9, 6, 1, 9, 5, 7, 3, 1, 3, 2, 4, 6, 4, 8, 8, 8, 1, 4, 5, 5, 3, 5, 3, 1, 0, 2, 8, 6, 5, 9, 7, 5, 3, 1, 0, 0, 8, 8, 2, 8, 1, 9, 1, 0, 1, 4, 6, 2, 0, 1, 2, 5, 8, 9, 4, 5, 2, 8, 4, 8, 0, 0, 3, 7, 1, 3, 0, 7, 7, 0, 3, 8, 4, 4, 2, 8, 2, 2, 9, 9, 4, 3, 3, 0, 0, 4, 5, 8, 3, 6, 5, 2, 0, 6, 6, 5, 3, 7, 1, 6, 0, 5, 2, 1, 6, 6, 7, 4, 5, 0, 0, 9, 6, 6, 3, 9, 7, 9, 9, 9, 6, 0, 3, 5, 6, 1, 4, 9, 5, 4, 8, 3, 5, 1, 8, 3, 9, 0, 2, 0, 9, 2, 5, 3, 1, 0, 9, 9, 3, 7, 3, 2, 1, 9, 1, 2, 8, 2, 3, 2, 5, 3, 0, 2, 1, 1, 5, 3, 6, 2, 1, 7, 5, 9, 8, 1, 1, 3, 4, 6, 7, 3, 8, 5, 8, 5, 2, 8, 7, 8, 5, 6, 9, 8, 5, 0, 5, 4, 5, 9, 7, 3, 2, 9, 4, 3, 9, 4, 5, 9, 7, 3, 9, 6, 4, 2, 7, 0, 0, 0, 4, 4, 5, 8, 1, 2, 4, 9, 5, 8, 8, 2, 7, 2, 4, 0, 0, 8, 9, 7, 5, 1, 0, 2, 1, 4, 0, 0, 3, 4, 8, 7, 9, 1, 5, 9, 9, 3, 7, 2, 9, 8, 3, 9, 9, 6, 5, 6, 8, 2, 7, 7, 4, 7, 2, 6, 6, 7, 6, 2, 2, 9, 8, 1, 5, 4, 7, 5, 4, 1, 6, 1, 6, 8, 1, 2, 7, 0, 5, 6, 6, 5, 8, 4, 9, 3